# QOL 

In [1]:
#Imports
import cv2
import mediapipe as mp
import pyautogui
import subprocess
import psutil
import time

In [2]:
#Programa principal

class App:
    def __init__(self):
        self.screen_size = pyautogui.size()
        self.screen_center = (self.screen_size[0]/2, self.screen_size[1]/2)
        self.window_size = (960,720)
        self.window_center = (self.window_size[0]/2, self.window_size[1]/2)

    def translate_coords(self, x, y, window):
        
        if window:
            aux = self.window_size
        else:
            aux = self.screen_size

        screen_x = int(x * aux[0])
        screen_y = int(y * aux[1])
        return screen_x, screen_y

    def dot_distance(self, t1, t2):
        dist = ((t2[0] - t1[0])**2 + (t2[1] - t1[1])**2)**0.5
        return dist

    def check_process(self, process):
        for proc in psutil.process_iter(['name']):
            try:
                if process in proc.info['name']:
                    print(True)
                    return True
            except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
                pass
        print(False)
        return False

    def close_process(self, process):
        for proc in psutil.process_iter(['name']):
            try:
                process_name = proc.info['name']
                if process in process_name:
                    proc.kill()
                    proc.wait()
                    return True
            except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
                pass
        return False    

    def video_play(self):
        # Inicializamos pyautogui
        pyautogui.FAILSAFE = False

        # Inicializar MediaPipe
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
        mp_draw = mp.solutions.drawing_utils

        # Input Camera
        vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        
        # Flag para el cambio de funcionalidad de la mano izquierda
        flag = True
        # Contador de frames
        frame_count = 0
        # Contador procesos
        proc_counter = 0
        # Flag para tratamiento de procesos
        proc_switch = True
        # Flag Auxiliar Macros
        macro_aux = True

        # Bucle de ejecución
        while True:
            # Fotograma de la cámara
            ret, frame = vid.read()
            if not ret:
                break
            
            frame_count += 1

            # Saltar fotogramas para mejorar rendimiento (cada 2 fotogramas)
            if frame_count % 2 != 0:
                continue
            
            # Cambiar tamaño del fotograma
            frame = cv2.resize(frame, (self.window_size[0], self.window_size[1]))
            # Voltear el fotograma
            frame = cv2.flip(frame, 1)
            h, w, _ = frame.shape  

            # Convertir el color BGR a RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Procesar el fotograma
            result = hands.process(rgb_frame)

            # Procesamiento
            if result.multi_hand_landmarks:
                for idx, hand_landmarks in enumerate(result.multi_hand_landmarks):
                    # Dibujar los puntos clave de la mano en la imagen
                    mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    # Obtener la información de la mano (derecha o izquierda)
                    handedness = result.multi_handedness[idx].classification[0].label

                    # Obtenemos los landmark 4(pulgar) , 8(indice), 12(corazón o medio), 16(anular), 20(menique) y 0 base de la palma
                    dedo_pulgar = hand_landmarks.landmark[4] 
                    dedo_indice = hand_landmarks.landmark[8]
                    dedo_medio = hand_landmarks.landmark[12]
                    dedo_anular = hand_landmarks.landmark[16]
                    dedo_menique = hand_landmarks.landmark[20]
                    base_palma = hand_landmarks.landmark[0]

                    # Pasamos a coordenadas de la pantalla
                    x_pulgar, y_pulgar = self.translate_coords(dedo_pulgar.x, dedo_pulgar.y, True)
                    x_indice, y_indice = self.translate_coords(dedo_indice.x, dedo_indice.y, True)       
                    x_medio, y_medio = self.translate_coords(dedo_medio.x, dedo_medio.y, True)
                    x_anular, y_anular = self.translate_coords(dedo_anular.x, dedo_anular.y, True)
                    x_menique, y_menique = self.translate_coords(dedo_menique.x, dedo_menique.y, True)
                    x_palma, y_palma = self.translate_coords(base_palma.x, base_palma.y, True)

                    # Calcular escala de la mano y el umbral para el clic
                    escala_mano = self.dot_distance((x_palma, y_palma), (x_indice, y_indice))
                    umbral_click = 0.2 * escala_mano # Ajustar el umbral

                    # Si la mano es derecha:
                    if handedness == 'Right':
                        # Movemos el raton con el dedo índice
                        # Mapeamos las coordenadas a la pantalla
                        screen_x_indice, screen_y_indice = self.translate_coords(dedo_indice.x, dedo_indice.y, False)
                        # Marcamos y movemos el cursor a la posición del dedo índice
                        cv2.circle(frame, (x_indice, y_indice), 10, (0, 255, 0), cv2.FILLED)
                        # Movemos el cursor a la posición del dedo índice
                        pyautogui.moveTo(screen_x_indice, screen_y_indice)

                        # Comprobamos si los dedos están cerca para hacer clic
                        # Calcular la distancia euclidiana entre el pulgar y el corazon
                        distancia_pulgar_medio = self.dot_distance((x_pulgar, y_pulgar), (x_medio, y_medio))
                        distancia_pulgar_anular = self.dot_distance((x_pulgar, y_pulgar), (x_anular, y_anular))
                        distancia_pulgar_menique = self.dot_distance((x_pulgar, y_pulgar), (x_menique, y_menique))

                        # Si la distancia es menor que un umbral, hacemos clic

                        if distancia_pulgar_menique < umbral_click:
                            if flag == True:
                                flag = False
                            else:
                                flag = True 
                            print(flag)  

                        if flag == True:
                            if distancia_pulgar_medio < umbral_click:
                                # Macro 1: Abrir y cerrar teclado virtual
                                if proc_switch == True:
                                    if self.check_process("osk.exe"):            
                                        self.close_process("osk.exe")
                                        print("Teclado virtual cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        subprocess.Popen("osk.exe", shell=True)
                                        print("Teclado virtual abierto")
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                            elif distancia_pulgar_anular < umbral_click:
                                # Macro 2 Abrir y cerrar Calculadora
                                if proc_switch == True:
                                    if self.check_process("CalculatorApp.exe"):            
                                        self.close_process("CalculatorApp.exe")
                                        print("Calculadora cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        subprocess.Popen("calc.exe", shell=True)
                                        print("Calculadora abierto")
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                        else:
                            if distancia_pulgar_medio < umbral_click:
                                # Macro 3: Abrir y cerrar Paint
                                if proc_switch == True:
                                    if self.check_process("mspaint.exe"):            
                                        self.close_process("mspaint.exe")
                                        print("Paint cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        subprocess.Popen("mspaint.exe", shell=True)
                                        print("Paint abierto")
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                            elif distancia_pulgar_anular < umbral_click:
                                # Macro 4 Abre Chrome y busca ulpgc.es
                                if proc_switch == True:
                                    if self.check_process("chrome.exe"):
                                        # Para cerrar chrome hay que mantenerlo ya que es del tipo de apps que abren multitud de procesos            
                                        self.close_process("chrome.exe")
                                        print("Chrome cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        # Si el programa no esta en la variable PATH, hay que poner la ruta completa
                                        subprocess.Popen(r"C:\Program Files\Google\Chrome\Application\chrome.exe", shell=True)
                                        time.sleep(1)
                                        print("Chrome abierto")
                                        pyautogui.write("Vision por Computador")
                                        time.sleep(1)
                                        pyautogui.press("enter")
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True

                    # Si la mano es izquierda:
                    elif handedness == 'Left':
                        # Comprobamos si los dedos están cerca para hacer clic
                        # Calcular la distancia euclidiana entre el pulgar y el corazon
                        distancia_pulgar_indice = self.dot_distance((x_pulgar, y_pulgar), (x_indice, y_indice))
                        distancia_pulgar_medio = self.dot_distance((x_pulgar, y_pulgar), (x_medio, y_medio))
                        distancia_pulgar_anular = self.dot_distance((x_pulgar, y_pulgar), (x_anular, y_anular))
                        distancia_pulgar_menique = self.dot_distance((x_pulgar, y_pulgar), (x_menique, y_menique))

                        # Comprobamos si el flag esta activado
                        if flag == True:
                            # Si la distancia es menor que un umbral, hacemos clic
                            if distancia_pulgar_indice < umbral_click:
                                pyautogui.click()
                            elif distancia_pulgar_medio < umbral_click:
                                pyautogui.click(button='right')
                            elif distancia_pulgar_anular < umbral_click:
                                # Hacer Scroll hacia arriba por una cantidad fija 
                                if proc_switch == True:
                                    pyautogui.scroll(30)             
                                    proc_switch = False
                                    proc_counter = frame_count
                                elif proc_counter + 5 < frame_count:
                                    proc_switch = True
                            elif distancia_pulgar_menique < umbral_click:
                                # Hacer Scroll hacia abajo por una cantidad fija 
                                if proc_switch == True:
                                    pyautogui.scroll(-30)             
                                    proc_switch = False
                                    proc_counter = frame_count
                                elif proc_counter + 5 < frame_count:
                                    proc_switch = True
                        else:
                            # Si la distancia es menor que un umbral, hacemos clic
                            if distancia_pulgar_indice < umbral_click:
                                # Para mantener el click izquierdo
                                pyautogui.mouseDown()
                            elif distancia_pulgar_medio < umbral_click:
                                # Macro 5 Abre el Bloc de notas y activa la dicción por voz
                                if proc_switch == True:
                                    if self.open_process("Notepad.exe"):            
                                        pyautogui.hotkey('esc')
                                        time.sleep(1)
                                        self.close_process("Notepad.exe")
                                        print("notepad cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        # Si el programa no esta en la variable PATH, hay que poner la ruta completa
                                        subprocess.Popen("notepad.exe", shell=True)
                                        time.sleep(1)
                                        print("notepad abierto")
                                        pyautogui.hotkey('win', 'h')
                                        time.sleep(1)
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                            elif distancia_pulgar_anular < umbral_click:
                                # Macro 6 Hacer un Ctrl + C y al siguiente clic un Ctrl + V
                                if proc_switch == True:
                                    if macro_aux == True:
                                        pyautogui.hotkey('ctrl', 'c')
                                        print("copiar")
                                        macro_aux = False
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        pyautogui.hotkey('ctrl', 'v')
                                        print("pegar")
                                        macro_aux = True
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                            elif distancia_pulgar_menique < umbral_click:
                                # Macro 7 Abre el Chrome y pone un video de musica para terminar
                                if proc_switch == True:
                                    if self.check_process("chrome.exe"):
                                        subprocess.run("taskkill /f /im chrome.exe", shell=True)
                                        print("Chrome cerrado")
                                        proc_switch = False
                                        proc_counter = frame_count
                                    else:
                                        # Si el programa no esta en la variable PATH, hay que poner la ruta completa
                                        subprocess.Popen([r"C:\Program Files\Google\Chrome\Application\chrome.exe", "https://www.youtube.com/watch?v=tiorwW7B-9Y"], shell=True)
                                        time.sleep(1)
                                        print("Chrome abierto")
                                        pyautogui.click(x=(self.screen_center[0]), y=(self.screen_center[1]))
                                        proc_switch = False
                                        proc_counter = frame_count
                                elif proc_counter + 15 < frame_count:
                                    proc_switch = True
                            
                            
            # Muestra fotograma
            cv2.imshow('Vid', frame)

            # Detenemos pulsado ESC
            if cv2.waitKey(20) == 27:
                break
            
        # Libera el objeto de captura
        vid.release()
        # Destruye ventanas
        cv2.destroyAllWindows()



In [3]:
app = App()

app.video_play()

False
Calculadora abierto
False
Teclado virtual abierto
True
Teclado virtual cerrado
True
Calculadora cerrado
False
True
False
True
False
False
Chrome abierto
True
False
True
False
Calculadora abierto
True
Calculadora cerrado
False
True
Chrome cerrado
True
Chrome cerrado
True
Chrome cerrado
True
Chrome cerrado
